# 07: Action Recommendation & Execution ⚡

This notebook implements the **Action Layer** (Day 6) of the SalesOps Agent Suite.

Until now, we have been *analyzing* data. Now, we **act** on it. This notebook demonstrates how an AI Agent can safely trigger downstream workflows (like creating Jira tickets or sending emails) while adhering to strict enterprise safety standards.

### 🎯 Goals
1.  **Start Mock Enterprise Environment:** Launch a local FastAPI server (`tools/mock_server.py`) to simulate Jira, Email, and CRM systems.
2.  **Load Insights:** Import the `enriched_anomalies.json` containing the AI's explanations and suggested actions.
3.  **Plan Actions:** Use the `ActionAgent` to translate insights into executable plans (determining priority, routing, and payloads).
4.  **Execute with Safety:** Run the actions against the mock API, demonstrating:
    * **Idempotency:** Preventing duplicate tickets.
    * **Retries:** Handling transient network failures.
    * **Audit Logging:** Recording every attempt in `../outputs/actions/actions.jsonl`.

### 🏗️ Components Used
* `agents.action_agent.ActionAgent`: The decision engine and API client.
* `tools.mock_server`: A FastAPI app simulating external enterprise services.

## 1: Imports

In [1]:
import sys
import os
import json
import time
import pandas as pd
import requests
from subprocess import Popen

# Add project root
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

os.environ["OBSERVABILITY_DIR"] = os.path.join(project_root, "outputs", "observability")

from agents.action_agent import ActionAgent

print("✅ Action Agent Loaded")

✅ Action Agent Loaded


## 2: Start Mock Server

In [2]:
import sys
import os
import time
import requests
from subprocess import Popen

# Get absolute path to project root
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))

print(f"🚀 Starting Mock Server from: {project_root}")
log_file = open("../outputs/mock_server.log", "w")

# 1. Use sys.executable -> ensures we use the active Conda environment
# 2. Run "-m uvicorn" -> safer than calling uvicorn directly
# 3. cwd=project_root -> ensures Python can find 'tools.mock_server'
process = Popen(
    [sys.executable, "-m", "uvicorn", "tools.mock_server:app", "--port", "7777"],
    stdout=log_file,
    stderr=log_file,
    cwd=project_root,
)

time.sleep(5)  # Wait a bit longer for startup on Windows

# Check Health & Reset Chaos
try:
    # Reset Chaos Config to Clean State
    requests.post(
        "http://localhost:7777/admin/chaos", json={"enabled": False, "failure_rate": 0}
    )

    resp = requests.get("http://localhost:7777/health")
    print("✅ Server Status:", resp.json())
except Exception as e:
    print("❌ Server failed to start.")

    # Read the log file to see what went wrong
    log_file.flush()
    with open("../outputs/mock_server.log", "r") as f:
        print("\n--- Server Log Output ---")
        print(f.read())

🚀 Starting Mock Server from: d:\01. Github\salesops-suite
✅ Server Status: {'status': 'healthy', 'port': 7777, 'uptime_since': '2025-11-29T13:28:11.885736+00:00', 'db_records': 9, 'db_path': 'outputs\\mock_db.json', 'config': {'chaos_enabled': False, 'failure_rate': 0.0, 'simulate_rate_limit': False}}


## 3: Load Data

In [3]:
INPUT_FILE = "../outputs/anomalies/enriched_anomalies.json"

with open(INPUT_FILE, "r") as f:
    anomalies = json.load(f)

print(f"Loaded {len(anomalies)} anomalies.")

Loaded 3 anomalies.


## 4: Run Agent with Custom Configuration

In [4]:
# We demonstrate that we can tune the agent without changing code (12-Factor App principles)

os.environ["MAX_RETRIES"] = "5"
os.environ["RETRY_BACKOFF"] = "0.5"

agent = ActionAgent()
print(
    f"Agent Configured: Max Retries={agent.MAX_RETRIES}, Backoff={agent.RETRY_BACKOFF}s"
)

# Run on top 5
results = agent.run_batch(anomalies[:5])

print(f"✅ Executed {len(results)} actions.")

Agent Configured: Max Retries=3, Backoff=1.0s
✅ Executed 6 actions.


## 5: Audit Log Inspection

In [5]:
# View the audit log
log_path = "../outputs/actions/actions.jsonl"
print(f"--- Audit Log ({log_path}) ---")

with open(log_path, "r") as f:
    for line in f:
        entry = json.loads(line)
        status = entry["result"]["status"]
        print(
            f"[{status.upper()}] {entry['type']} -> {entry['result'].get('response', 'No Response')}"
        )

--- Audit Log (../outputs/actions/actions.jsonl) ---
[SUCCESS] create_ticket -> {'ticket_id': 'TICKET-43077', 'status': 'created', 'link': 'https://jira.internal/browse/TICKET-43077', 'review_url': None, 'created_at': '2025-11-29T06:21:20.324046+00:00'}
[SUCCESS] create_ticket -> {'ticket_id': 'TICKET-43077', 'status': 'created', 'link': 'https://jira.internal/browse/TICKET-43077', 'review_url': None, 'created_at': '2025-11-29T06:21:20.324046+00:00'}
[SUCCESS] create_ticket -> {'ticket_id': 'TICKET-22405', 'status': 'created', 'link': 'https://jira.internal/browse/TICKET-22405', 'review_url': None, 'created_at': '2025-11-29T06:21:24.440334+00:00'}
[SUCCESS] create_ticket -> {'ticket_id': 'TICKET-22405', 'status': 'created', 'link': 'https://jira.internal/browse/TICKET-22405', 'review_url': None, 'created_at': '2025-11-29T06:21:24.440334+00:00'}
[SUCCESS] create_ticket -> {'ticket_id': 'TICKET-18256', 'status': 'created', 'link': 'https://jira.internal/browse/TICKET-18256', 'review_url'

## 6: Cleanup

In [6]:
# Kill the server process
process.terminate()
log_file.close()
print("🛑 Server Stopped.")

🛑 Server Stopped.


## ⏭️ Next Step: The Grand Orchestration

Success! We have officially "Closed the Loop."

We have built a complete vertical slice of an autonomous organization:
1.  **Ingestion:** Loading raw data.
2.  **Detection:** Finding statistical outliers.
3.  **Explanation:** Using GenAI to understand "Why".
4.  **Action:** Triggering enterprise workflows to fix the problem.

**However, currently, we are the "Orchestrator."** We are manually running these notebooks one by one.

In the final development phase (**Day 7**), we will build the **A2A (Agent-to-Agent) Coordinator**. We will use the **Sequential Agent** pattern to link all these independent specialists into a single, autonomous pipeline that can run from start to finish with a single command.

👉 **Proceed to `notebooks/08_orchestration.ipynb`.**